
# View Lucid Dreaming Core Session Data

This notebook loads and visualizes data recorded by the `app/main.py` application.
It reads the `session_metadata.npz` file for session information and then loads the corresponding
`eeg_eog_data.dat` and `aux_sensor_data.dat` binary files.


In [3]:

import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime

# Configure matplotlib for inline plotting
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid') # Using a seaborn style for better aesthetics


In [4]:

# --- Configuration & Session Selection ---
BASE_RECORDING_FOLDER = "app/recorded_data"
METADATA_FILENAME = "session_metadata.npz"
EEG_EOG_DATA_FILENAME = "eeg_eog_data.dat"
AUX_SENSOR_DATA_FILENAME = "aux_sensor_data.dat"
EEG_DATA_TYPE = np.float32 # Should match what's in main.py

def list_session_folders(base_folder):
    if not os.path.isdir(base_folder):
        print(f"Error: Base recording folder not found: {base_folder}")
        return []
    sessions = sorted([d for d in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, d))], reverse=True)
    return sessions

available_sessions = list_session_folders(BASE_RECORDING_FOLDER)
if not available_sessions:
    print(f"No session folders found in {BASE_RECORDING_FOLDER}")
else:
    print("Available sessions:")
    for i, session_id in enumerate(available_sessions):
        print(f"{i}: {session_id}")
    
    # Select a session (e.g., the latest one)
    selected_session_index = 0 # Or use input() for user to choose
    if 0 <= selected_session_index < len(available_sessions):
        SESSION_FOLDER_NAME = available_sessions[selected_session_index]
        SESSION_PATH = os.path.join(BASE_RECORDING_FOLDER, SESSION_FOLDER_NAME)
        print(f"\nSelected session: {SESSION_FOLDER_NAME}")
        print(f"Session path: {SESSION_PATH}")
    else:
        print("Invalid session index selected.")
        SESSION_PATH = None

Available sessions:
0: 20250520_230403_168461
1: 20250520_223611_155605

Selected session: 20250520_230403_168461
Session path: app/recorded_data/20250520_230403_168461


In [5]:

# --- Load Metadata --- 
metadata = None
if SESSION_PATH and os.path.isdir(SESSION_PATH):
    metadata_filepath = os.path.join(SESSION_PATH, METADATA_FILENAME)
    if os.path.exists(metadata_filepath):
        try:
            metadata = np.load(metadata_filepath, allow_pickle=True)
            print("\n--- Session Metadata ---")
            for key, value in metadata.items():
                if key not in ['scores', 'eeg_eog_data_info', 'aux_sensor_data_info'] and not key.startswith("metadata_"):
                    print(f"{key}: {value}")
            
            if 'eeg_eog_data_info' in metadata:
                print("\nEEG/EOG Data Info:")
                for k,v in metadata['eeg_eog_data_info'].item().items(): # .item() if it's a 0-d array object
                    print(f"  {k}: {v}")
            if 'aux_sensor_data_info' in metadata:
                print("\nAuxiliary Sensor Data Info:")
                for k,v in metadata['aux_sensor_data_info'].item().items():
                    print(f"  {k}: {v}")
            
            # Store for later use
            eeg_eog_info = metadata['eeg_eog_data_info'].item() if 'eeg_eog_data_info' in metadata else None
            aux_info = metadata['aux_sensor_data_info'].item() if 'aux_sensor_data_info' in metadata else None
            sampling_frequency = metadata['sampling_frequency_hz'].item() if 'sampling_frequency_hz' in metadata else 250.0

        except Exception as e:
            print(f"Error loading metadata file {metadata_filepath}: {e}")
            metadata = None
    else:
        print(f"Metadata file not found: {metadata_filepath}")
else:
    print("Session path not set or invalid.")



--- Session Metadata ---
product_key: RUtYA4W3kpXi0i9C7VZCQJY5_GRhm4XL2rKp6cviwQI=
device_id: FRENZI40
session_start_iso: 2025-05-20T23:04:03.168461
sampling_frequency_hz: 125.0

EEG/EOG Data Info:
  filename: eeg_eog_data.dat
  data_type: float32
  channel_names: ['RAW_EEG_LF', 'RAW_EEG_OTEL', 'RAW_EEG_REF1', 'RAW_EEG_RF', 'RAW_EEG_OTER', 'RAW_EEG_REF2', 'FILT_EEG_LF', 'FILT_EEG_OTEL', 'FILT_EEG_RF', 'FILT_EEG_OTER', 'FILT_EOG_CH1', 'FILT_EOG_CH2', 'FILT_EOG_CH3', 'FILT_EOG_CH4']
  num_channels: 14
  shape_on_save: channels_first

Auxiliary Sensor Data Info:
  filename: aux_sensor_data.dat
  data_type: float32
  channel_names: ['FILT_EMG_CH1', 'FILT_EMG_CH2', 'FILT_EMG_CH3', 'FILT_EMG_CH4', 'RAW_IMU_X', 'RAW_IMU_Y', 'RAW_IMU_Z', 'RAW_PPG_GREEN', 'RAW_PPG_RED', 'RAW_PPG_IR']
  num_channels: 10
  shape_on_save: channels_first


In [6]:

# --- Function to Load .dat Files ---
def load_dat_file(filepath, num_channels, total_samples, dtype=np.float32):
    if not os.path.exists(filepath):
        print(f"Data file not found: {filepath}")
        return None
    try:
        data_flat = np.fromfile(filepath, dtype=dtype)
        # Expected number of elements
        expected_elements = num_channels * total_samples
        if data_flat.size != expected_elements:
            print(f"Warning: File size mismatch for {filepath}. Expected {expected_elements} elements, got {data_flat.size}.")
            # Attempt to reshape with actual elements, might lead to incorrect total_samples if file is corrupt/incomplete
            # For robust handling, one might need to adjust total_samples or num_channels based on data_flat.size
            # For now, we'll try to reshape with the number of samples that fits the channel count
            if data_flat.size % num_channels == 0:
                actual_total_samples = data_flat.size // num_channels
                if actual_total_samples != total_samples:
                    print(f"Adjusting total samples for {filepath} from {total_samples} to {actual_total_samples} based on file size.")
                total_samples = actual_total_samples
            else:
                print(f"Error: Cannot reshape data for {filepath} as size {data_flat.size} is not divisible by num_channels {num_channels}.")
                return None
        
        # Reshape to (num_channels, total_samples)
        # Data was written as (channels, samples_in_block).tobytes(), so it's C-contiguous.
        reshaped_data = data_flat.reshape(num_channels, total_samples)
        return reshaped_data
    except Exception as e:
        print(f"Error loading or reshaping data file {filepath}: {e}")
        return None


In [7]:

# --- Load EEG/EOG Data ---
eeg_eog_data_loaded = None
if metadata and eeg_eog_info and SESSION_PATH:
    eeg_eog_data_filepath = os.path.join(SESSION_PATH, EEG_EOG_DATA_FILENAME)
    num_eeg_eog_channels = eeg_eog_info['num_channels']
    
    if 'metadata_eeg_eog_sample_counts' in metadata:
        total_eeg_eog_samples = np.sum(metadata['metadata_eeg_eog_sample_counts'])
        if total_eeg_eog_samples > 0:
            eeg_eog_data_loaded = load_dat_file(eeg_eog_data_filepath, num_eeg_eog_channels, total_eeg_eog_samples, dtype=EEG_DATA_TYPE)
            if eeg_eog_data_loaded is not None:
                print(f"\nLoaded EEG/EOG data. Shape: {eeg_eog_data_loaded.shape}")
        else:
            print("No EEG/EOG samples recorded according to metadata.")
    else:
        print("EEG/EOG sample counts not found in metadata.")



Loaded EEG/EOG data. Shape: (14, 294875)


In [8]:
# --- Print First 5 Rows of EEG/EOG Data ---
if eeg_eog_data_loaded is not None:
    print("First 5 rows of EEG/EOG data (channels x samples):")
    print(eeg_eog_data_loaded[:4, :])
else:
    print("EEG/EOG data not loaded.")


First 5 rows of EEG/EOG data (channels x samples):
[[ 9.32000000e+03  9.32000000e+03  9.32000000e+03 ...  1.46802094e+02
   1.69247894e+02  2.04758133e+02]
 [ 3.51603622e+01  4.00326157e+01  4.51104889e+01 ...  4.65912390e+00
   7.08482504e+00  8.07183456e+00]
 [ 5.84500000e+03  5.83500000e+03  5.81400000e+03 ... -1.23507051e+01
  -5.23574305e+00 -8.50970459e+00]
 [-3.38537521e+01 -3.34189301e+01 -3.42980423e+01 ... -1.10153055e+01
  -8.93185520e+00 -6.67639923e+00]]


In [23]:

# --- Load Auxiliary Sensor Data ---
aux_data_loaded = None
if metadata and aux_info and SESSION_PATH:
    aux_data_filepath = os.path.join(SESSION_PATH, AUX_SENSOR_DATA_FILENAME)
    num_aux_channels = aux_info['num_channels']
    
    if 'metadata_aux_sample_counts' in metadata:
        total_aux_samples = np.sum(metadata['metadata_aux_sample_counts'])
        if total_aux_samples > 0:
            aux_data_loaded = load_dat_file(aux_data_filepath, num_aux_channels, total_aux_samples, dtype=EEG_DATA_TYPE)
            if aux_data_loaded is not None:
                print(f"\nLoaded Auxiliary sensor data. Shape: {aux_data_loaded.shape}")
        else:
            print("No Auxiliary sensor samples recorded according to metadata.")
    else:
        print("Auxiliary sensor sample counts not found in metadata.")



Loaded Auxiliary sensor data. Shape: (10, 5000)


In [24]:
# --- Print First 5 Rows of Auxiliary Sensor Data ---
if aux_data_loaded is not None:
    print("First 5 rows of Auxiliary sensor data (channels x samples):")
    print(aux_data_loaded[:5, :])
else:
    print("Auxiliary sensor data not loaded.")


First 5 rows of Auxiliary sensor data (channels x samples):
[[-1.4412411e+00 -1.5017254e+00 -7.0405960e-01 ...            nan
             nan            nan]
 [ 2.1190817e+00  9.8768854e+00  4.3505722e+01 ...            nan
             nan            nan]
 [-1.3380608e+01  1.9752497e+01 -3.8103056e+00 ...  2.0807900e+05
   2.0813000e+05  2.0813600e+05]
 [-9.1413260e-01  9.8136514e-01  1.3064108e+00 ...  2.0907100e+05
   2.0909100e+05  2.0908300e+05]
 [ 3.1863903e+01  2.2856979e+01  2.3144464e+01 ...  2.0838300e+05
   2.0840900e+05  2.0842600e+05]]


In [20]:

# --- Display Scores ---
if metadata and 'scores' in metadata:
    scores_dict = metadata['scores'].item() # .item() because it's saved as a 0-d object array containing the dict
    print("\n--- Saved Scores ---")
    for score_name, score_array in scores_dict.items():
        print(f"\nScore: {score_name} (Length: {len(score_array) if hasattr(score_array, '__len__') else 'N/A'})")
        print(score_array)
        
        # Example: Plot sleep stage if available and not empty
        if score_name == "array__sleep_stage" and hasattr(score_array, '__len__') and len(score_array) > 0:
            plt.figure(figsize=(12, 3))
            # Assuming scores are roughly every 30s for sleep stage, create a simple x-axis
            # For more accuracy, one would need timestamps per score point if available
            score_time_axis = np.arange(len(score_array))
            plt.plot(score_time_axis, score_array, marker='o', linestyle='-')
            plt.title("Sleep Stages Over Session")
            plt.xlabel("Segment (e.g., 30s epoch number)")
            plt.ylabel("Sleep Stage")
            # value < 0: undefined, value = 0: awake, value = 1: light, value = 2: deep, value = 3: REM
            plt.yticks([-1, 0, 1, 2, 3], ['Undefined', 'Awake', 'Light', 'Deep', 'REM'])
            plt.grid(True)
            plt.show()
else:
    print("Scores not found in metadata.")



--- Saved Scores ---

Score: array__sleep_stage (Length: 0)
[]

Score: array__poas (Length: 0)
[]

Score: array__posture (Length: 7)
['upright' 'upright' 'upright' 'upright' 'upright' 'upright' 'upright']

Score: array__focus_score (Length: 20)
[40.         40.         40.         40.         40.         40.
 40.81724336 42.44151009 44.38123499 47.90894181 51.98287235 55.98949534
 58.93907686 60.47720593 60.29954905 58.80419991 56.29855942 53.1963579
 50.08986352 47.67902808]

Score: array__sqc_scores (Length: 7)
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
